# Assignment 4.1

In [1]:
import time
import socket
import multiprocessing

In [2]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

In [3]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"
#include "timer.h"
#include "stdint.h"

void pwm(unsigned int per_us, unsigned int pin, unsigned int t){
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, 0);
    
    int numLoops = (1000000*t)/per_us;
    
    for(int i = 0; i < numLoops; i++)
    {
        gpio_write(pin_out, 1);
        delay_us(per_us);
        gpio_write(pin_out, 0);
        delay_us(per_us);
    }

}


## Server code
The serer code runs a server that will run forever, until the client send a kill message. Fist the Server defines some constant values for the HOST_IP, PORT, and messages it is expecting ot recieve. Next the server creates a socket and bonds it to the HOST_IP (aka IP-address of client board) with the defined PORT. 

An inner function if defined to play the buzzer tone. This function takes the pwm freqiency, pwm generator, and play time as arguments. The pwm_generator should be of type `Pmod_PWM`, and the tone is generated by simply calling the `Pmod_PWM.generate(per_us, dc)` function then sleeping for the passed play time before stopping the pwm generator.

Pmod_PMN Doc: https://pynq.readthedocs.io/en/v2.1/pynq_package/pynq.lib/pynq.lib.pmod.html#module-pynq.lib.pmod.pmod_pwm

The main Server code consits of two nested while loops. The outer loop runs as long `running_server` remains `True`. It listens for the client and waits for a connection. Once a connection is established the `client_connected` variable is set to `True`. The inner loop runs as long as `client_connected = True`. It first waits to receive a message from the client. Once a message is received the value of the message is compared to the known messages. If the `SOUND_ALARM_MSG` is received then the server will run the `play_alarm` message. If the `DISCONNECT_MSG` is received then the `client_connected` variable gets set to `False`, thus breaking the inner loop and returning to the top of the outer while loop to wait for a new connection. If the `KILL_SERVER_MSG` is recieved then the `running_server` and `client_connected` variables both get set to `False` which breaks both loops and ends the server process. 

In [4]:
#the server runs on its own process and listens for the 'sound_alarm' message.
#once the message is received, the server plays the buzzer alarm. 
#the client can also send a kill message to stop the server (to get the jupyter cell to stop).
def server(alarm_freq, play_time=1):
    HOST_IP = '127.0.0.1' #local host
    PORT = 3334
    SOUND_ALARM_MSG, DISCONNECT_MSG = 'sound_alarm', 'disconnecting'
    KILL_SERVER_MSG = 'kill_server'
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    # 1: Bind the socket to the pynq board <HOST-IP> at port <LISTENING-PORT>
    sock.bind((HOST_IP, PORT))

    #Inner function to play alarm for passed play_time
    def play_alarm(freq, play_time):
        per_us = int((1/freq)*1e6)
        print('Period: {} us, Freq: {} hz'.format(round(per_us,1), freq))
        pwm(per_us,2,play_time)
        
    running_server = True
    while running_server:
        print('Server Trying to Listen')
        sock.listen(1)
        print("Server listening...")
        conn, addr = sock.accept()
        with conn:
            print('Connected by', addr)
            client_connected = True
            while client_connected:
                msg = conn.recv(2048).decode()
                if msg == SOUND_ALARM_MSG:
                    print('PLAYING ALARM')
                    tt = time.time()
                    play_alarm(alarm_freq, play_time)
                    print('DONE PLAYING ALARM FOR {} SEC'.format(round(time.time()-tt,2)))
                elif msg == DISCONNECT_MSG:
                    print('client disconnected...')
                    client_connected = False
                elif msg== KILL_SERVER_MSG:
                    print('kill order receieved from client..')
                    client_connected = False
                    running_server = False
                    sock.close()
                else:
                    print('Unrecognized Message')
    print('SERVER KILLED')

## Client Code
The Client code runs a client process that listens for button presses and sends messages to the server depending on which buttons were pressed. First the client defines constants for the `SERVER_IP` and `PORT`. Then constants for the button presses are defined on line 3. Lines 4 and 5 define the `SOUND_ALARM_MSG`, `DISCONNECT_MSG`, and `KILL_SERVER_MSG` constants, and must exactly match the messages that the server expects to receive. Finally the Debounce time is set to be slightly longer than the buzzer play time, to stop the client from sending `SOUND_ALARM_MSG`'s faster than the server can play them. 

The clinet process has an inner function named `run_client()` that creates a new socket and connects the socket to the server. Then once connected the `running_client` variable is set to `True` so that a while loop can continuously poll for button presses using the `base.btns_gpio.read()` function. If `Button 1` on the board is pressed then the `SOUND_ALARM_MSG` is sent to the server. If `Button 1` is pressed, the `DISCONNECT_MSG` is sent to the server, the socket is closed, the `running_clinet` variable is set to false to break the while loop, and the function returns `True`. If `Button 3` is pressed then the `KILL_SERVER_MSG` is sent to the server, the socket is closed, and the function returns `False`.

The bottom chuck of code for the clinet process is a whil loop that runs forever as long `client_process_running = True`. This loop repeatedly checks if `Button 0` is pressed. If it is pressed then the `run_client()` function is called. If `run_client()` returns `True` then the loop continues and the client can be reconnected by pressing `Button 0` again. If `run_clinet()` returns `False` then the kill button was pressed so the loop ends and client process joins. 

In [5]:
def client(btns):
    SERVER_IP, PORT = '127.0.0.1', 3334
    RUN_CLIENT, SEND_MSG, DISCONNECT, KILL = 0b0001, 0b0010, 0b0100, 0b1000
    SOUND_ALARM_MSG, DISCONNECT_MSG = 'sound_alarm', 'disconnecting'
    KILL_SERVER_MSG = 'kill_server'
    DEBOUNCE_TIME = 1.1
    
    def run_client():
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.connect((SERVER_IP,PORT))
        running_client = True
        while running_client:
            if btns.read() == SEND_MSG:
                print('client sending message')
                sock.sendall(SOUND_ALARM_MSG.encode())
                time.sleep(DEBOUNCE_TIME)
            elif btns.read() == DISCONNECT:
                print('client disconnecting')
                sock.sendall(DISCONNECT_MSG.encode())
                sock.close()
                running_client = False
            elif btns.read() == KILL:
                print('kill sent from client')
                sock.sendall(KILL_SERVER_MSG.encode())
                sock.close()
                return False    
            else: pass #not a button press we care about
        return True
    
    #inifinte loop
    client_process_running = True
    while client_process_running:
        if btns.read() == RUN_CLIENT:
            print('client connecting')
            client_process_running = run_client()     
    print('CLIENT KILLED\n')

## main function
THe main function simply sets up constants for the buzzer frequency and play time. Create a Pmos_PWM object to be used by the server to generate pwm wavforms on the defined pin number. Seperate processed are created for the server and the client so that they can run independantly of each other. Both processed are started and will run forever, unless `Button 3`(the kill button) is pressed while the server and the client are connected. 

In [6]:
def main():
    ALARM_FREQ = 2500
    ALARM_TIME = 1
    PIN_NUM = 2
    BUTTONS = base.btns_gpio
    p1 = multiprocessing.Process(target=server, args=(ALARM_FREQ, ALARM_TIME)) 
    p1.start() # start the process
    time.sleep(0.2)
    p2 = multiprocessing.Process(target=client, args=(BUTTONS,)) 
    p2.start() # start the process
    p1.join()
    print('p1 joined')
    p2.join()
    print('p2 joined')

In [7]:
main()

Server Trying to Listen
Server listening...
client connecting
Connected by ('127.0.0.1', 37062)
client sending message
PLAYING ALARM
Period: 400 us, Freq: 2500 hz
DONE PLAYING ALARM FOR 0.03 SEC
client sending message
PLAYING ALARM
Period: 400 us, Freq: 2500 hz
DONE PLAYING ALARM FOR 0.02 SEC
client sending message
PLAYING ALARM
Period: 400 us, Freq: 2500 hz
DONE PLAYING ALARM FOR 0.02 SEC
client disconnecting
client disconnected...
Server Trying to Listen
Server listening...
client connecting
Connected by ('127.0.0.1', 37140)
client sending message
PLAYING ALARM
Period: 400 us, Freq: 2500 hz
DONE PLAYING ALARM FOR 0.02 SEC
client sending message
PLAYING ALARM
Period: 400 us, Freq: 2500 hz
DONE PLAYING ALARM FOR 0.02 SEC
client sending message
PLAYING ALARM
Period: 400 us, Freq: 2500 hz
DONE PLAYING ALARM FOR 0.03 SEC
client disconnecting
client disconnected...
Server Trying to Listen
Server listening...
client connecting
Connected by ('127.0.0.1', 37218)
kill sent from client
CLIENT K

## Simple Example For passing buttons into a process

In [ ]:
def hello(buttons):
    while True:
        if buttons.read() != 0:
            print(bin(buttons.read()))
p1 = multiprocessing.Process(target=hello, args=(base.btns_gpio,)) 
p1.start() # start the process
p1.join()